# Jetpack Compose - ROOM - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu lokalnej bazy danych **ROOM** w aplikacji.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExam82azljYWxraXAwcDM3dWk1NnowMTYwbDFqY3lzcDB1bm1uZHQxNCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/wJ4PVlB2HlfUf2giVm/giphy.gif" width="200" />

`Room` jest to biblioteka dostarczona przez Android Jetpack, która ułatwia pracę z lokalną bazą danych `SQLite`. Stanowi wygodny i wydajny sposób na przechowywanie danych wewnętrznych aplikacji. `SQLite` to lekka, wbudowana baza danych, która jest szeroko stosowana w aplikacjach mobilnych i innych systemach osadzonych.

Główne cechy `SQLite` to:
- Działa jako biblioteka dostępna w postaci pliku w aplikacji, co oznacza, że nie wymaga oddzielnego procesu serwera bazy danych. Aplikacja może bezpośrednio komunikować się z bazą danych.
- Jest samowystarczalna, co oznacza, że cała baza danych jest przechowywana w jednym pliku. Nie ma potrzeby konfigurowania lub zarządzania wieloma plikami lub zasobami.
- Transakcje - Obsługuje transakcje, co umożliwia grupowanie operacji bazodanowych jako pojedyncze, atomowe działanie. Transakcje są ważne, gdy chodzi o utrzymanie integralności danych. Każda operacja na bazie jest wykonywana w całości lub w ogóle - jest to ważne przy modyfikowaniu wielu tabel, oraz asynchronicznym przetwarzaniu.
- Wsparcie dla standardowego `SQL` - obsługuje większość standardowego języka `SQL`, co ułatwia programowanie i wykonywanie zapytań.
- Jest zaprojektowany w taki sposób, aby działał wydajnie nawet na urządzeniach o ograniczonych zasobach sprzętowych.

Główne składniki biblioteki `Room` to:
- `Entity` - Reprezentuje tabelę w bazie danych `SQLite`. Każda klasa oznaczona adnotacją `@Entity` może być mapowana do jednej tabeli w bazie danych, a pola klasy odpowiadają kolumnom tej tabeli.
- `DAO` (*Data Access Object*) - Definiuje metody, które umożliwiają dostęp do danych w bazie danych. Możemy zdefiniować interfejs `DAO` za pomocą adnotacji `@Dao`, a Room automatycznie dostarczy implementację tych metod.
- `Database` - Klasa bazowa, która reprezentuje bazę danych. To miejsce, gdzie definiujemy wszystkie *encje*, które mają zostać użyte w aplikacji, oraz wersję bazy danych. Room tworzy implementację bazy danych opartej na tej klasie.

Aplikacja będzie wyświetlać listę wszystkich użytkowników przechowywanych w bazie danych na urządzeniu. Dodamy operacje dodawania pojedynczego użytkownika oraz czyszczenia danych z bazy.

Dodajmy niezbędne zależności i pluginy do plików konfiguracyjnych aplikacji

```kotlin
plugins {
    ...
    id 'kotlin-kapt'
}
...
dependencies {

    implementation "androidx.lifecycle:lifecycle-viewmodel-compose:2.6.1"
    implementation "androidx.lifecycle:lifecycle-viewmodel-ktx:2.6.1"
    implementation "androidx.lifecycle:lifecycle-runtime-compose:2.6.1"

    implementation "androidx.room:room-runtime:2.5.2"
    annotationProcessor "androidx.room:room-compiler:2.5.2"
    kapt "androidx.room:room-compiler:2.5.2"
    implementation "androidx.room:room-ktx:2.5.2"
    ...
}
```

### **UWAGA!!!**
Istnieje nowsza wtyczka służąca do przetwarzania adnotacji - `ksp`, jednak w trakcie pisania niniejszego notatnika jest w fazie testów i jeszcze jest niestabilna. Z tego powodu w notatnikach dalej będzie wykorzystywana starsza wtyczka `kapt`. `ksp` wprowadzono w celu zastąpienia `kapt`, więc w przyszłości może być konieczna migracja, bądź powyższy blok `plugins` będzie zdezaktualizowany. Więcej informacji [tutaj](https://developer.android.com/build/migrate-to-ksp)

- **KAPT (Kotlin Annotation Processing Tool)** - jest to wtyczka kompilatora używana do przetwarzania adnotacji w języku Kotlin. Współpracuje z bibliotekami, które korzystają z adnotacji do generowania kodu w czasie kompilacji. Jej głównym zadaniem jest skanowanie kodu źródłowego w poszukiwaniu adnotacji i generowanie dodatkowego kodu na podstawie ich wystąpień. Jest to szczególnie użyteczne w przypadku bibliotek, które chcą automatycznie generować kod dla określonych przypadków użycia.
- **KSP (Kotlin Symbol Processing)** - to nowsza wtyczka kompilatora, wprowadzona w celu **zastąpienia** `kapt`. Również służy do przetwarzania adnotacji i generowania kodu w czasie kompilacji. Główną różnicą jest jednak to, że `ksp` zostało zaprojektowane z myślą o bardziej elastycznym i wydajnym API. Oferuje programistom dostęp do bardziej zaawansowanego drzewa symboli, co ułatwia przetwarzanie i analizę kodu aplikacji. Dzięki temu może działać szybciej i jest bardziej skalowalne niż starsza wersja.

Będziemy dodawać użytkowników, więc dodajmy obiekt pomocniczy, który ułatwi ich generowanie.

In [ ]:
data class User(
    val firstName: String,
    val lastName: String
)

In [ ]:
object DataProvider {
    private val firstNames = listOf(
        "Adam", "Ewa", "Jan", "Anna", "Piotr", "Maria", "Tomasz", "Małgorzata", "Krzysztof", "Alicja",
        "Andrzej", "Joanna", "Michał", "Barbara", "Kamil", "Magdalena", "Robert", "Monika", "Mateusz", "Natalia"
    )

    private val lastNames = listOf(
        "Nowak", "Kowalski", "Wiśniewski", "Wójcik", "Kowalczyk", "Kamiński", "Lewandowski", "Zieliński", "Szymański",
        "Woźniak", "Dąbrowski", "Kozłowski", "Jankowski", "Mazur", "Kwiatkowski", "Krawczyk", "Piotrowski", "Grabowski",
        "Nowakowski", "Pawłowski"
    )

    val user: User
        get() = User(firstNames.random(), lastNames.random())
}

Przy każdej próbie odczytu pola `user` zostanie wywołany *getter*, który utworzy nową instancję klasy `User`

## Elementy Room

Przejdźmy do dodania wszystkich elementów bazy danych, rozpoczniemi od `Entity`. Zmodyfikujmy wcześniej dodaną klasę `User`.

In [ ]:
@Entity(tableName = "user_table")
data class User(
    @PrimaryKey(autoGenerate = true) val id: Int,
    val firstName: String,
    val lastName: String
)

- `@Entity(tableName = "user_table")` - Adnotacja `@Entity` jest używana do oznaczenia klasy jako *encji* bazy danych. Oznacza to, że obiekty tej klasy będą reprezentować wiersze w tabeli bazy danych. `tableName` to atrybut adnotacji `@Entity`, który definiuje nazwę tabeli, do której będą mapowane obiekty tej klasy. W tym przypadku tabela będzie miała nazwę `"user_table"`.
- `@PrimaryKey(autoGenerate = true) val id: Int` - Adnotacja `@PrimaryKey` informuje, że pole `id` będzie kluczem głównym tabeli. W bazie danych, pole oznaczone jako klucz główny musi być unikatowe dla każdego wiersza. `autoGenerate = true` oznacza, że wartość klucza będzie generowana automatycznie przy dodawaniu nowego rekordu do tabeli.

Adnotacja `@Entity` może posiadać szereg innych atrybutów:
- `indices` - Pozwala na zdefiniowanie indeksów dla jednego lub wielu pól w tabeli. Indeksy pomagają w przyspieszeniu wyszukiwania danych w bazie danych.
```kotlin
@Entity(tableName = "user_table", indices = [Index(value = ["firstName", "lastName"])])
data class User(
    // ...
)
```
- `foreignKeys` - Pozwala na zdefiniowanie kluczy obcych w tabeli. Określa relacje między tabelą bieżącą a innymi tabelami w bazie danych.
```kotlin
@Entity(
    tableName = "order_table",
    foreignKeys = [
        ForeignKey(
            entity = User::class,
            parentColumns = ["id"],
            childColumns = ["user_id"],
            onDelete = ForeignKey.CASCADE
        )
    ]
)
data class Order(
    // ...
    val user_id: Int
)
```
- `primaryKeys` - Pozwala na zdefiniowanie niestandardowego zestawu pól jako kluczy głównych tabeli.
```kotlin
@Entity(tableName = "book_table", primaryKeys = ["isbn", "title"])
data class Book(
    val isbn: String,
    val title: String,
    // ...
)
```
- `ignoredColumns` - Pozwala na zdefiniowanie pól, które będą ignorowane przez `Room` i nie będą mapowane do tabeli w bazie danych.
```kotlin
@Entity(tableName = "user_table", ignoredColumns = ["age"])
data class User(
    val firstName: String,
    val lastName: String,
    val age: Int // ignorowane
    // ...
)
```

Kolejnym elementem będzie dodanie interfejsu `DAO`

In [ ]:
@Dao
interface UserDao {
    @Query("SELECT * FROM user_table ORDER BY lastName ASC, firstName ASC")
    fun getUsers(): Flow<List<User>>

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    suspend fun insert(user: User)

    @Query("DELETE FROM user_table")
    suspend fun deleteAll()
}

Interfejs `DAO` zawiera deklaracje metod, które pozwalają na interakcję z bazą danych i wykonywanie operacji.

- `@Query("SELECT * FROM user_table ORDER BY lastName ASC, firstName ASC")` - Jest to metoda do pobierania użytkowników z bazy danych. Zapytanie `SQL` `SELECT * FROM user_table ORDER BY lastName ASC, firstName ASC` wybiera wszystkie kolumny z tabeli `"user_table"` i sortuje wyniki według kolumny `lastName` rosnąco, a następnie według kolumny `firstName` rosnąco. Ta metoda zwraca dane jako `Flow<List<User>>`, co oznacza, że może przesyłać nowe wyniki w czasie rzeczywistym (np. gdy dane się zmienią).
- `@Insert(onConflict = OnConflictStrategy.IGNORE)` - Jest to metoda do dodawania użytkownika do bazy danych. Adnotacja `@Insert` oznacza, że ta metoda jest używana do wstawiania danych do tabeli. Atrybut `onConflict = OnConflictStrategy.IGNORE` oznacza, że jeśli próba wstawienia użytkownika zakończy się konfliktem (np. duplikatem klucza głównego), zostanie zignorowana.
- `@Query("DELETE FROM user_table")` - Jest to metoda do usuwania wszystkich użytkowników z bazy danych. Zapytanie `SQL` `DELETE FROM user_table` usuwa wszystkie wiersze z tabeli `"user_table"` i czyszczenie jej zawartości.

- `@Insert` - specjalna adnotacja, która nie wymaga podawania wprost kodu `SQL` - istanieją jeszcze `@Delete` oraz `@Update` - mocno ułatwia to wykonywanie podstawowych operacji na bazie. Adnotacja `@Query` wymaga podania pełnego zapytania `SQL` jako `String`

Lista niektórych adnotacji, które można wykorzystać:
- `@Update` - służy do aktualizacji danych w bazie danych. Metody oznaczone tą adnotacją muszą przyjmować jako argumenty obiekty klas, które reprezentują wiersze w tabeli. `Room` automatycznie wygeneruje odpowiednie zapytania `SQL` do zaktualizowania tych danych.
```kotlin
@Dao
interface UserDao {
    // ...

    @Update
    suspend fun update(user: User)

    // ...
}
```
- `@Delete` - jest używana do usuwania danych z bazy danych. Metody oznaczone tą adnotacją powinny przyjmować jako argumenty obiekty klas reprezentujących wiersze, które mają zostać usunięte.
```kotlin
@Dao
interface UserDao {
    // ...

    @Delete
    suspend fun delete(user: User)

    // ...
}
```
- `@Query` - pozwala na definiowanie własnych zapytań `SQL`.
```kotlin
@Dao
interface UserDao {
    // ...

    @Query("SELECT * FROM user_table WHERE firstName = :firstName") // zwraca listę wszystkich użytkowników gdzie pole w bazie (firstName) 
                                                                    // jest równe argumentowi przekazanemu w funkcji (:firstName)
    fun getUsersByFirstName(firstName: String): List<User>

    // ...
}
```
- `@RawQuery` - pozwala na wykonywanie niezdefiniowanych z góry zapytań `SQL`, które można przekazać jako argument typu `SupportSQLiteQuery`.
```kotlin
@Dao
interface UserDao {
    // ...

    @RawQuery
    fun getUsersByRawQuery(query: SupportSQLiteQuery): List<User>

    // ...
}
```
- `@Transaction` - używana jest do oznaczenia metod, które wymagają wykonania kilku metod w transakcji. Transakcje pozwalają na wykonywanie zestawu operacji jako pojedynczą, atomową operację - albo transakcja jest wykonana w pełni, albo wcale.
```kotlin
@Dao
interface UserDao {
    // ...

    @Transaction
    suspend fun insertAndUpdate(user: User, updatedUser: User) {
        insert(user)
        update(updatedUser)
    }

    // ...
}
```

Nie implementujemy metod tego interfejsu, ponieważ to `Room` automatycznie generuje implementacje tych metod w czasie kompilacji. Koncepcja ta nazywa się *automatyczną implementacją*.

`Room` wykorzystuje mechanizm **Proxy** w języku Kotlin (lub **Refleksję** w języku Java), aby analizować interfejs `DAO` i generować kod `SQL` oraz operacje na bazie danych zgodnie z adnotacjami i deklaracjami metod. Dzięki temu nie musimy ręcznie implementować tych metod i zapytań.

Ostatnim elementem będzie utworzenie klasy bazowej.

In [ ]:
@Database(entities = [User::class], version = 1, exportSchema = false)
abstract class UserDatabase : RoomDatabase() {
    abstract fun userDao(): UserDao

    companion object {
        @Volatile
        private var Instance: UserDatabase? = null

        fun getDatabase(context: Context): UserDatabase {
            return Instance ?: synchronized(this) {
                Room.databaseBuilder(context, UserDatabase::class.java, "user_database")
                    .build()
                    .also { Instance = it }
            }
        }
    }
}

Klasa ta reprezentuje bazę danych dla aplikacji, która zawiera tylko jedną tabelę `User`. Wykorzystuje wzorzec **Singleton** i synchronizację w celu zapewnienia, że baza danych będzie miała tylko jedną instancję w całej aplikacji.

- `@Database(entities = [User::class], version = 1, exportSchema = false)` - Adnotacja `@Database` jest używana do oznaczenia klasy `UserDatabase` jako bazowej klasy bazy danych `Room`.
    - `entities` definiuje tablicę klas encji, które będą zawarte w tej bazie danych. 
    - `version` definiuje numer wersji bazy danych.
    - `exportSchema` określa, czy `Room` ma eksportować schemat bazy danych do pliku na urządzeniu. W tym przypadku jest ustawiony na false - migracje bazy są poza zakresem przedmiotu.
- `abstract fun userDao(): UserDao` - Jest to deklaracja abstrakcyjnej metody, która zwraca obiekt o typie interfejsu `UserDao`. Ta metoda będzie używana do uzyskania dostępu do operacji bazodanowych związanych z encją `User`. Jest metodą abstrakcyjną, ponieważ nie ma zdefiniowanej implementacji. Faktyczna implementacja tej metody jest generowana automatycznie przez `Room` w czasie kompilacji.
- `@Volatile private var Instance: UserDatabase? = null` - Zmienna `Instance` jest oznaczona jako `volatile`, co zapewnia, że jej wartość jest zawsze widoczna dla innych wątków.
- `fun getDatabase(context: Context): UserDatabase` - Jest to metoda fabryczna, która tworzy lub zwraca istniejącą instancję bazy danych. Jeśli instancja bazy danych już istnieje, zostanie zwrócona; w przeciwnym razie zostanie utworzona za pomocą `Room.databaseBuilder()`.
- `Room.databaseBuilder()` używa `context` aplikacji, klasy `UserDatabase` oraz nazwy bazy danych (`"user_database"`) do skonfigurowania i zbudowania instancji `UserDatabase`. 
- `Metoda synchronized()` jest używana do synchronizacji dostępu do kodu, co zapewnia, że tylko jeden wątek może uzyskać dostęp do tej sekcji jednocześnie.

## ViewModel

Przejdźmy do utworzenia klasy `UserViewModel`, ponieważ `UserDatabase` wymaga podania kontekstu aplikacji, przekażemy go przez konstruktor. Jest to problem, ponieważ `ViewModel` nie przyjmuje żadnych parametrów, aby to zmienić musimy zaimplementować własną fabrykę.

In [ ]:
class UserViewModelFactory(val application: Application) :
    ViewModelProvider.Factory {
    override fun <T : ViewModel> create(modelClass: Class<T>): T {
        return UserViewModel(application) as T
    }
}

- `val application: Application` - Jest to pole przechowujące obiekt klasy Application, który jest dostępny w całej aplikacji i reprezentuje kontekst aplikacji. Przekazywanie obiektu `Application` do `UserViewModelFactory` pozwala na jego wykorzystanie do tworzenia instancji `UserViewModel`, a w efekcie do utworzenia instancji `UserDatabase`.
- `ViewModelProvider.Factory` - Jest to interfejs, który definiuje metodę `create`, która jest odpowiedzialna za tworzenie instancji `ViewModel`. W praktyce, korzystając z `ViewModelProvider` i `ViewModelProvider.Factory`, możemy przekazywać dodatkowe argumenty do `ViewModel`'ów, które są wymagane do ich konstrukcji (na przykład obiekt bazy danych, repozytorium, inny ViewModel itp.).
- `override fun <T : ViewModel> create(modelClass: Class<T>): T` Jest to metoda, która tworzy i zwraca instancję `ViewModel`. Parametr `modelClass` wskazuje na klasę `ViewModel`, który ma być utworzony.
- `return UserViewModel(application) as T` Jest to instrukcja, która tworzy instancję `UserViewModel` i zwraca ją jako obiekt typu `T`. `T` jest typem dziedziczącym z `ViewModel`, więc rzutujemy `UserViewModel` na `T`.

Następnie utwórzmy właściwy `UserViewModel`

In [ ]:
class UserViewModel(application: Application) : ViewModel() {

    private val repository: UserRepository
    private val _usersState = MutableStateFlow<List<User>>(emptyList())
    val usersState: StateFlow<List<User>>
        get() = _usersState

    init {
        val db = UserDatabase.getDatabase(application)
        val dao = db.userDao()
        repository = UserRepository(dao)

        fetchUsers()
    }

    private fun fetchUsers() {
        viewModelScope.launch {
            repository.getUsers().collect { users ->
                _usersState.value = users
            }
        }
    }

    fun clearUsers() {
        viewModelScope.launch {
            repository.clear()
        }
    }

    fun addUser(user: User) {
        viewModelScope.launch {
            repository.add(user)
        }
    }
}

- `init { ... }` blok inicjalizacyjny, który wykonuje kod podczas tworzenia instancji `UserViewModel`. W tym przypadku, następuje utworzenie instancji `UserDatabase`, uzyskanie `DAO` z bazy danych, a także utworzenie i zainicjowanie repozytorium `UserRepository`. Pozostałe elementy są znane z poprzednich przykładów.

### **UWAGA!!!**
Tego typu tworzenie obiektów `UserDatabase` i `UserRepository` jest problematyczne - w ostatnim module zapoznamy się z techniką wstrzykiwania zależności (*dependency injection*), która pozwoli nam rozwiązać ten problem.

## ui

Dodajmy funckję renderującą ekran główny aplikacji.

In [ ]:
@Composable
fun MainScreen(){

    val viewModel: UserViewModel = viewModel(
        LocalViewModelStoreOwner.current!!,
        "UserViewModel",
        UserViewModelFactory(LocalContext.current.applicationContext as Application)
    )
    val users by viewModel.usersState.collectAsStateWithLifecycle()

    Column(modifier = Modifier.fillMaxSize()) {
        LazyColumn(
            modifier = Modifier.fillMaxWidth().weight(0.7f)
        ) {
            items(users.size) {
                Text(
                    text = "${users[it].firstName} ${users[it].lastName}",
                    fontSize = 32.sp,
                    textAlign = TextAlign.Center,
                    modifier = Modifier
                        .fillMaxWidth()
                        .padding(2.dp)
                )
            }
        }

        Button(
            onClick = { viewModel.addUser(DataProvider.user) },
            modifier = Modifier.fillMaxWidth().padding(4.dp)
        ) {
            Text(text = "ADD")
        }

        Button(
            onClick = { viewModel.clearUsers() },
            modifier = Modifier.fillMaxWidth().padding(4.dp)
        ) {
            Text(text = "CLEAR")
        }
    }
}

W powyższym kodzie jedynym nowym elementem jest uzyskanie instancji `UserViewModel`

In [ ]:
val viewModel: UserViewModel = viewModel(
    LocalViewModelStoreOwner.current!!,
    "UserViewModel",
    UserViewModelFactory(LocalContext.current.applicationContext as Application)
)

Chcemy skorzystać z zaimplementowanej wcześniej fabryki - funkcja `viewModel` przyjmuje kilka argumentów:

- viewModelStoreOwner: ViewModelStoreOwner - Parametr określa właściciela `ViewModelStore`, do którego `ViewModel` będzie przywiązany. `ViewModelStore` to miejsce przechowywania `ViewModel` i ich stanu między rekompozycjami.
- `key: String` - Jest unikalnym kluczem, który identyfikuje `ViewModel` wewnątrz `ViewModelStore`.
- `factory: () -> T` - Tworzy instancję ViewModelu. Jest to sposób na odroczenie tworzenia `ViewModel` do momentu, gdy jest rzeczywiście potrzebny. Funkcja `factory` nie przyjmuje żadnych argumentów i zwraca instancję `ViewModel` (`T`). Gdy `viewModel()` jest wywoływane w środowisku `Compose`, funkcja `factory` zostanie wykonana tylko raz i instancja `ViewModel` będzie przechowywana i dostępna do ponownego użycia między rekompozycjami.

Przekazane parametry:
- `LocalViewModelStoreOwner.current!!` - `LocalViewModelStoreOwner` to zmienna lokalna, która dostarcza obiekt `ViewModelStoreOwner`. W tym przypadku wykorzystujemy `LocalViewModelStoreOwner.current!!`, co oznacza, że `ViewModel` jest przywiązany do bieżącego właściciela `ViewModelStoreOwner` w drzewie `Compose`.
- `"UserViewModel"` - Jest to unikatowy klucz, który identyfikuje `ViewModel` w `ViewModelStore`.
- `UserViewModelFactory(LocalContext.current.applicationContext as Application)` - Jest to instancja `UserViewModelFactory`, która dostarcza zależność (`Application`). W ten sposób `UserViewModel` może uzyskać dostęp do kontekstu aplikacji, co jest niezbędne do uzyskania instancji bazy danych.

Funkcję wywołujemy w aktywności głównej aplikacji.

In [ ]:
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContent {
            ROOMBasicsComposeTheme {
                // A surface container using the 'background' color from the theme
                Surface(
                    modifier = Modifier.fillMaxSize(),
                    color = MaterialTheme.colorScheme.background
                ) {
                    MainScreen()
                }
            }
        }
    }
}

Możemy przetestować aplikacjię.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExam82azljYWxraXAwcDM3dWk1NnowMTYwbDFqY3lzcDB1bm1uZHQxNCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/wJ4PVlB2HlfUf2giVm/giphy.gif" width="200" />